In [ ]:
! nvidia-smi

Tue May 24 07:52:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os

class Config:
    AUTHOR = "kuruton"

    NAME = "USP-" + "Exp094-funnel-transformer-large-base-kf-gauss"
    MODEL_PATH = "funnel-transformer/large-base"
    DATASET_PATH = [
        "yasufuminakama/cpc-data"
    ]

    COMPETITION = "us-patent-phrase-to-phrase-matching"
    COLAB_PATH = "/content/drive/Shareddrives/USPatent" 
    DRIVE_PATH = os.path.join(COLAB_PATH, AUTHOR)

    api_path = "/content/drive/MyDrive/kaggle.json"

    seed = 42
    num_fold = 4
    trn_fold = [0, 1, 2, 3]
    batch_size = 32
    n_epochs = 50
    max_len = 256

    weight_decay = 2e-5
    beta = (0.9, 0.98)
    lr = 2e-5
    num_warmup_steps_rate = 0.01
    clip_grad_norm = None
    gradient_accumulation_steps = 1
    num_eval = 3

    upload_from_colab = True

In [ ]:
# ========================================
# Library
# ========================================
import os
import gc
import re
import sys
import json
import time
import shutil
import joblib
import random
import requests
import warnings
warnings.filterwarnings('ignore')
from ast import literal_eval

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy 
import itertools
from pathlib import Path
from glob import glob
from tqdm.auto import tqdm
from sklearn.model_selection import (
    StratifiedKFold, 
    KFold, 
    GroupKFold
)
from sklearn.metrics import (
    accuracy_score, 
    f1_score,
    roc_auc_score,
)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.cuda.amp import autocast, GradScaler

from google.colab import drive
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
def setup(cfg):
    cfg.COLAB = 'google.colab' in sys.modules
    cfg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if cfg.COLAB:
        print('This environment is Google Colab')

        # mount
        from google.colab import drive
        if not os.path.isdir('/content/drive'):
            drive.mount('/content/drive') 

        # pip install
        # ! pip install -q torch==1.10.0
        ! pip install -q transformers
        ! pip install -q sentencepiece

        # use kaggle api (need kaggle token)
        f = open(cfg.api_path, 'r')
        json_data = json.load(f) 
        os.environ['KAGGLE_USERNAME'] = json_data['username']
        os.environ['KAGGLE_KEY'] = json_data['key']

        # set dirs
        cfg.DRIVE = cfg.DRIVE_PATH
        cfg.EXP = (cfg.NAME if cfg.NAME is not None 
            else requests.get('http://172.28.0.2:9000/api/sessions').json()[0]['name'][:-6]
        )
        cfg.INPUT = os.path.join(cfg.DRIVE, 'Input')
        cfg.OUTPUT = os.path.join(cfg.DRIVE, 'Output')
        cfg.SUBMISSION = os.path.join(cfg.DRIVE, 'Submission')
        cfg.DATASET = os.path.join(cfg.DRIVE, 'Dataset')

        cfg.OUTPUT_EXP = os.path.join(cfg.OUTPUT, cfg.EXP) 
        cfg.EXP_MODEL = os.path.join(cfg.OUTPUT_EXP, 'model')
        cfg.EXP_FIG = os.path.join(cfg.OUTPUT_EXP, 'fig')
        cfg.EXP_PREDS = os.path.join(cfg.OUTPUT_EXP, 'preds')

        # make dirs
        for d in [cfg.INPUT, cfg.SUBMISSION, cfg.EXP_MODEL, cfg.EXP_FIG, cfg.EXP_PREDS]:
            os.makedirs(d, exist_ok=True)
        
        if not os.path.isfile(os.path.join(cfg.INPUT, 'train.csv')):
            # load dataset
            ! pip install --upgrade --force-reinstall --no-deps kaggle
            ! kaggle competitions download -c $cfg.COMPETITION -p $cfg.INPUT
            filepath = os.path.join(cfg.INPUT,cfg.COMPETITION+'.zip')
            ! unzip -d $cfg.INPUT $filepath
            
        
        for path in cfg.DATASET_PATH:
            datasetpath = os.path.join(cfg.DATASET,  path.split('/')[1])
            if not os.path.exists(datasetpath):
                os.makedirs(datasetpath, exist_ok=True)
                ! kaggle datasets download $path -p $datasetpath
                filepath = os.path.join(datasetpath, path.split("/")[1]+'.zip')
                ! unzip -d $datasetpath $filepath

    else:
        print('This environment is Kaggle Kernel')

        # set dirs
        cfg.INPUT = f'../input/{cfg.COMPETITION}'
        cfg.EXP = cfg.NAME
        cfg.OUTPUT_EXP = cfg.NAME
        cfg.SUBMISSION = './'
        cfg.DATASET = '../input/'
        
        cfg.EXP_MODEL = os.path.join(cfg.EXP, 'model')
        cfg.EXP_FIG = os.path.join(cfg.EXP, 'fig')
        cfg.EXP_PREDS = os.path.join(cfg.EXP, 'preds')

        # make dirs
        for d in [cfg.EXP_MODEL, cfg.EXP_FIG, cfg.EXP_PREDS]:
            os.makedirs(d, exist_ok=True)
    return cfg


def dataset_create_new(dataset_name, upload_dir):
    dataset_metadata = {}
    dataset_metadata['id'] = f'{os.environ["KAGGLE_USERNAME"]}/{dataset_name}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = dataset_name
    with open(os.path.join(upload_dir, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode='tar')

In [ ]:
# =====================
# Utils
# =====================
# Seed
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# KFold
def get_kfold(train, n_splits, seed):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    generator = kf.split(train)
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

def get_stratifiedkfold(train, target_col, n_splits, seed):
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    generator = kf.split(train, train[target_col])
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

def get_groupkfold(train, target_col, group_col, n_splits):
    kf = GroupKFold(n_splits=n_splits)
    generator = kf.split(train, train[target_col], train[group_col])
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

# collatte
def collatte(inputs, labels=None):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    if not labels is None:
        inputs = {
            "input_ids" : inputs['input_ids'][:,:mask_len],
            "attention_mask" : inputs['attention_mask'][:,:mask_len],
        }
        labels =  labels[:,:mask_len]
        return inputs, labels, mask_len
                
    else:
        inputs = {
            "input_ids" : inputs['input_ids'][:,:mask_len],
            "attention_mask" : inputs['attention_mask'][:,:mask_len],
        }
        return inputs, mask_len

# =====================
# CPC Data
# =====================
def get_cpc_texts(cfg):
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir(os.path.join(cfg.DATASET, 'cpc-data/CPCSchemeXML202105')):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(os.path.join(cfg.DATASET, f'cpc-data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt')) as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = result[0].lstrip(pattern)
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            results[context] = cpc_result + ". " + result[0].lstrip(pattern)
    for key, val in results.items():
        results[key] = val.lower()
    return results


def get_optimizer_params(model, lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    layer_num = len(model.backbone.encoder.blocks)
    layer1_num = layer_num // 3
    layer2_num = layer_num // 3
    layer3_num = layer_num - (layer1_num + layer2_num)
    group1 = ['blocks.' + str(i) + '.' for i in range(layer1_num)]
    group2 = ['blocks.' + str(i) + '.' for i in range(layer1_num, layer1_num + layer2_num)]
    group3 = ['blocks.' + str(i) + '.' for i in range(layer1_num + layer2_num, layer1_num + layer2_num + layer3_num)]
    group_all = group1 + group2 + group3
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.backbone.named_parameters() if (not any(nd in n for nd in no_decay)) and (not any(nd in n for nd in group2)) and (not any(nd in n for nd in group3))] ,
          'lr': lr * 0.25, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.backbone.named_parameters() if any(nd in n for nd in no_decay) and (not any(nd in n for nd in group2)) and (not any(nd in n for nd in group3))],
          'lr': lr * 0.25, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if "backbone" not in n],
          'lr': lr * 0.2, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in group2) and (not any(nd in n for nd in no_decay))],
        'lr': lr * 0.5, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in group2) and any(nd in n for nd in no_decay)],
        'lr': lr * 0.5, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in group3) and (not any(nd in n for nd in no_decay))],
        'lr': lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in group3) and any(nd in n for nd in no_decay)],
        'lr': lr, 'weight_decay': 0.0}
    ]
    return optimizer_grouped_parameters

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values
        self.labels = df['score'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        inputs = self.prepare_input(self.cfg, self.texts[index])
        label = torch.tensor(self.labels[index], dtype=torch.float)
        return inputs, label
    
    @staticmethod
    def prepare_input(cfg, text):
        inputs = cfg.tokenizer(
            text,
            add_special_tokens=True,
            max_length=cfg.max_len,
            padding="max_length",
            return_offsets_mapping=False
        )
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long)
        return inputs

class CustomModel(nn.Module):
    def __init__(self, cfg, criterion):
        super().__init__()
        self.cfg = cfg
        self.criterion = criterion
        self.config = AutoConfig.from_pretrained(
            cfg.MODEL_PATH,
            output_hidden_states=True
        )
        self.config.attention_probs_dropout_prob = 0.0
        self.config.hidden_dropout_prob = 0.0
        self.backbone = AutoModel.from_pretrained(
            cfg.MODEL_PATH, 
            config=self.config
        )
        self.fc = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1),
        )
        self.softplus = nn.Softplus()
        self.var = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1),
        )

    def forward(self, inputs, labels=None):
        outputs = self.backbone(**inputs)["last_hidden_state"]
        outputs = outputs[:, 0, :]
        logits = self.fc(outputs).flatten()
        var = self.var(outputs).flatten()
        loss = 0
        if labels is not None:
            loss = self.criterion(logits, labels, self.softplus(var))
        return logits, loss


class CustomModelDropout(nn.Module):
    def __init__(self, cfg, criterion):
        super().__init__()
        self.cfg = cfg
        self.criterion = criterion
        self.config = AutoConfig.from_pretrained(
            cfg.MODEL_PATH,
            output_hidden_states=True
        )
        self.backbone = AutoModel.from_pretrained(
            cfg.MODEL_PATH, 
            config=self.config
        )
        self.fc = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1),
        )

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)

    def forward(self, inputs, labels=None):
        outputs = self.backbone(**inputs)["last_hidden_state"]
        outputs = outputs[:, 0, :]
        logits1 = self.fc(self.dropout1(outputs)).flatten()
        logits2 = self.fc(self.dropout2(outputs)).flatten()
        logits4 = self.fc(self.dropout3(outputs)).flatten()
        logits3 = self.fc(self.dropout4(outputs)).flatten()
        logits5 = self.fc(self.dropout5(outputs)).flatten()
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        
        loss = 0
        if labels is not None:
            loss1 = self.criterion(logits1, labels,)
            loss2 = self.criterion(logits2, labels)
            loss3 = self.criterion(logits3, labels)
            loss4 = self.criterion(logits4, labels)
            loss5 = self.criterion(logits5, labels)
            loss = (loss1 + loss2 + loss3 + loss4 + loss5) / 5

        return logits, loss

In [ ]:
# evaluating
def evaluating(cfg, valid_loader, model, criterion, valid_df, fold, best_val_preds, best_val_score):
    # evaluating
    val_preds = []
    val_losses = []
    val_nums = []
    model.eval()
    with torch.no_grad():
        with tqdm(valid_loader, total=len(valid_loader)) as pbar:
            for (inputs, labels) in pbar:
                for k, v in inputs.items():
                    inputs[k] = v.to(cfg.device)
                labels = labels.to(cfg.device)
                with autocast():
                    output, loss = model(inputs, labels)
                output = output.detach().cpu().numpy()
                val_preds.append(output)
                val_losses.append(loss.item() * len(labels))
                val_nums.append(len(labels))
                pbar.set_postfix({
                    'val_loss': loss.item()
                })

    val_preds = np.concatenate(val_preds)
    val_loss = sum(val_losses) / sum(val_nums)
    corr_score = np.corrcoef(val_preds, valid_df['score'])[0, 1]

    val_log = {
        'val_loss': val_loss,
        'score': corr_score,
    }
    display(val_log)

    if best_val_score < corr_score:
        print("save model weight")
        best_val_preds = val_preds
        best_val_score = corr_score
        torch.save(
            model.state_dict(), 
            os.path.join(cfg.EXP_MODEL, f"fold{fold}.pth")
        )
    return best_val_preds, best_val_score


def training(cfg, train):
    # =====================
    # Training
    # =====================
    set_seed(cfg.seed)
    oof_pred = np.zeros(len(train), dtype=np.float32)
    for fold in cfg.trn_fold:
        # dataset, dataloader
        train_df = train.loc[cfg.folds!=fold]
        valid_df = train.loc[cfg.folds==fold]
        train_idx = list(train_df.index)
        valid_idx = list(valid_df.index)

        # reverse anchor_target
        # rev_train_df = train_df.copy()
        # rev_train_df[['anchor', 'target']] = rev_train_df[['target', 'anchor']].to_numpy()
        # train_df = pd.concat([train_df, rev_train_df]).reset_index(drop=True)

        train_dataset = TrainDataset(cfg, train_df)
        valid_dataset = TrainDataset(cfg, valid_df)
        train_loader = DataLoader(
            dataset=train_dataset, 
            batch_size=cfg.batch_size, 
            shuffle=True,
            pin_memory=True,
            drop_last=True
        )
        valid_loader = DataLoader(
            dataset=valid_dataset,
            batch_size=cfg.batch_size,
            shuffle=False,
            pin_memory=True,
            drop_last=False
        )
        # model-training
        # criterion = nn.BCEWithLogitsLoss(reduction="mean")
        criterion = nn.GaussianNLLLoss(reduction="mean")
        best_val_preds = None
        best_val_score = -1

        # model
        model = CustomModel(cfg, criterion)
        model = model.to(cfg.device)


        # optimizer, scheduler
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        # optimizer_grouped_parameters = [
        #     {
        #         'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
        #         'weight_decay': cfg.weight_decay
        #     },
        #     {
        #         'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
        #         'weight_decay': 0.0
        #     }
        # ]

        optimizer_grouped_parameters = get_optimizer_params(
            model,
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )

        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=cfg.lr,
            betas=cfg.beta,
            weight_decay=cfg.weight_decay,
        )
        num_train_optimization_steps = int(
            len(train_loader) * cfg.n_epochs // cfg.gradient_accumulation_steps
        )
        num_warmup_steps = int(num_train_optimization_steps * cfg.num_warmup_steps_rate)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_train_optimization_steps
        )
        num_eval_step = len(train_loader) // cfg.num_eval + cfg.num_eval
        
        for epoch in range(cfg.n_epochs):
            # training
            print(f"# ============ start epoch:{epoch} ============== #")
            model.train() 
            val_losses_batch = []
            scaler = GradScaler()
            with tqdm(train_loader, total=len(train_loader)) as pbar:
                for step, (inputs, labels) in enumerate(pbar):
                    inputs, max_len = collatte(inputs)
                    for k, v in inputs.items():
                        inputs[k] = v.to(cfg.device)
                    labels = labels.to(cfg.device)

                    optimizer.zero_grad()
                    with autocast():
                        output, loss = model(inputs, labels)
                    pbar.set_postfix({
                        'loss': loss.item(),
                        'lr': scheduler.get_lr()[0]
                    })

                    if cfg.gradient_accumulation_steps > 1:
                        loss = loss / cfg.gradient_accumulation_steps

                    scaler.scale(loss).backward()
                    if cfg.clip_grad_norm is not None:
                        torch.nn.utils.clip_grad_norm_(
                            model.parameters(), 
                            cfg.clip_grad_norm
                        )
                    if (step+1) % cfg.gradient_accumulation_steps == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        scheduler.step()

                    if step % num_eval_step == 0 and step != 0:
                        tmp_num_eval = step // num_eval_step
                        print(f'fold: {fold}, epoch: {epoch}, num_eval: {tmp_num_eval}')
                        best_val_preds, best_val_score = evaluating(
                            cfg, valid_loader,
                            model,
                            criterion,
                            valid_df,
                            fold,
                            best_val_preds,
                            best_val_score
                        )
                        model.train() 
            
            print(f'fold: {fold}, epoch: {epoch}, last')
            best_val_preds, best_val_score = evaluating(
                cfg, valid_loader,
                model,
                criterion,
                valid_df,
                fold,
                best_val_preds,
                best_val_score
            )

        oof_pred[valid_idx] = best_val_preds.astype(np.float32)
        np.save(os.path.join(cfg.EXP_PREDS, f'oof_pred_fold{fold}.npy'), best_val_preds)
        del model; gc.collect()

    np.save(os.path.join(cfg.EXP_PREDS, 'oof_pred.npy'), oof_pred)

    # =====================
    # scoring
    # =====================
    corr_score = np.corrcoef(oof_pred, train['score'])[0, 1]
    print('CV:', round(corr_score, 5))
    return corr_score

In [ ]:
cfg = setup(Config)
if cfg.upload_from_colab and cfg.COLAB:
    from kaggle.api.kaggle_api_extended import KaggleApi
    dataset_create_new(dataset_name=Config.EXP, upload_dir=Config.OUTPUT_EXP)

This environment is Google Colab
     |████████████████████████████████| 4.2 MB 14.3 MB/s 
     |████████████████████████████████| 596 kB 60.3 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 
     |████████████████████████████████| 1.2 MB 15.1 MB/s 
Starting upload for file model.tar


100%|██████████| 10.0k/10.0k [00:01<00:00, 5.73kB/s]


Upload successful: model.tar (10KB)
Starting upload for file fig.tar


100%|██████████| 10.0k/10.0k [00:02<00:00, 4.00kB/s]


Upload successful: fig.tar (10KB)
Starting upload for file preds.tar


100%|██████████| 10.0k/10.0k [00:02<00:00, 4.31kB/s]


Upload successful: preds.tar (10KB)


In [9]:
# =====================
# Main
# =====================
cfg = setup(Config)

import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

train = pd.read_csv(os.path.join(cfg.INPUT, 'train.csv'))
test = pd.read_csv(os.path.join(cfg.INPUT, 'test.csv'))
sub = pd.read_csv(os.path.join(cfg.INPUT, 'sample_submission.csv'))

cfg.tokenizer = AutoTokenizer.from_pretrained(cfg.MODEL_PATH)
print(cfg.tokenizer.sep_token)

cpc_texts = get_cpc_texts(cfg)
torch.save(cpc_texts, os.path.join(cfg.EXP_PREDS, "cpc_texts.pth"))
train['context_text'] = train['context'].map(cpc_texts)
train['text'] = train['anchor'] + cfg.tokenizer.sep_token + train['target'] + cfg.tokenizer.sep_token + train['context_text']

# cfg.folds = get_groupkfold(train, 'score', 'anchor', cfg.num_fold)
cfg.folds = get_kfold(train, cfg.num_fold, cfg.seed)
cfg.folds.to_csv(os.path.join(cfg.EXP_PREDS, 'folds.csv'))

score = training(cfg, train)
if cfg.upload_from_colab and cfg.COLAB:
    from kaggle.api.kaggle_api_extended import KaggleApi
    dataset_create_new(dataset_name=Config.EXP, upload_dir=Config.OUTPUT_EXP)

This environment is Google Colab


Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/705 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

{'score': 0.843566457529447, 'val_loss': 74.86270357951126}

CV: 0.8528
Starting upload for file model.tar


100%|██████████| 5.35G/5.35G [02:25<00:00, 39.4MB/s]


Upload successful: model.tar (5GB)
Starting upload for file fig.tar


ProtocolError: ignored